In [1]:
%pip install semantic-link --quiet 
import synapse.ml.core
import pandas as pd
from synapse.ml.services import *
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, flatten, udf, lower, trim
from pyspark.sql.types import StringType
import sempy
import sempy.fabric as fabric
import base64
import json
import copy

StatementMeta(, 0b843145-b532-4aca-9aa2-62d0ffcc7add, 8, Finished, Available)


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



# Parameters & initialize

In [2]:
# Parameters
workspaceId = fabric.get_workspace_id()
reportId = "054e3ecf-56d4-4da5-9610-af4148fc839a"

languages = ["fr", "it", "pt", "ja"]
#German-de; Hindi-hi; Hungarian-hu;Maori-mi;Thai-th

newReportName = "[trans] - Sales"

# Client initialization
client = fabric.FabricRestClient()

StatementMeta(, 0b843145-b532-4aca-9aa2-62d0ffcc7add, 10, Finished, Available)

# Cleanup for the demo

In [8]:
response = client.get(f"/v1/workspaces/{workspaceId}/items")

listItems = response.json()['value']

itemsToDelete = [x for x in listItems if x['type'] == 'Report' and x['displayName'].startswith(newReportName)]

for item in itemsToDelete:
    itemId = item['id']
    
    print(f"Deleting report '{itemId}'")

    client.delete(f"/v1/workspaces/{workspaceId}/items/{itemId}")

StatementMeta(, 0b843145-b532-4aca-9aa2-62d0ffcc7add, 16, Finished, Available)

Deleting report '683694a0-0b2c-464f-9e9e-c8752a7e7aec'
Deleting report 'bac3cf3f-75c1-4b23-8c53-0ab2b18967a4'
Deleting report 'c08a7b42-a327-4878-a85a-a70874c7a4b2'
Deleting report '47d71167-3467-416a-9dae-f2194dd744ce'


# Get report definition parts

In [4]:
response = client.post(f"/v1/workspaces/{workspaceId}/items/{reportId}/getDefinition")

jsonParts = response.json()['definition']['parts']

df_parts = pd.json_normalize(jsonParts)
pd.set_option('display.max_colwidth', 100) 
df_parts

StatementMeta(, 0b843145-b532-4aca-9aa2-62d0ffcc7add, 12, Finished, Available)

,path,payload,payloadType
0,definition.pbir,ewogICJ2ZXJzaW9uIjogIjQuMCIsCiAgImRhdGFzZXRSZWZlcmVuY2UiOiB7CiAgICAiYnlQYXRoIjogbnVsbCwKICAgICJi...,InlineBase64
1,StaticResources/RegisteredResources/Light4437032645752863.json,ewogICIkc2NoZW1hIjogImh0dHBzOi8vcmF3LmdpdGh1YnVzZXJjb250ZW50LmNvbS9taWNyb3NvZnQvcG93ZXJiaS1kZXNr...,InlineBase64
2,StaticResources/RegisteredResources/_7abfc6c7-1a23-4b5f-bd8b-8dc472366284171093267.jpg,77+9UE5HDQoaCgAAAA1JSERSAAACAAAAAgAIBgAAAO+/vXjvv73vv70AAAAJcEhZcwAACxMAAAsTAQDvv73vv70YAAAAAXNS...,InlineBase64
3,StaticResources/SharedResources/BaseThemes/CY23SU04.json,ewogICJuYW1lIjogIkNZMjNTVTA0IiwKICAiZGF0YUNvbG9ycyI6IFsKICAgICIjMTE4REZGIiwKICAgICIjMTIyMzlFIiwK...,InlineBase64
4,report.json,ewogICJjb25maWciOiAie1widmVyc2lvblwiOlwiNS40M1wiLFwidGhlbWVDb2xsZWN0aW9uXCI6e1wiYmFzZVRoZW1lXCI6...,InlineBase64


# Enumerate all the report visuals **with** titles

In [9]:
# Get the report json part and parse it

reportPart = [part for part in jsonParts if part["path"] == 'report.json'][0]

payload = reportPart['payload']

reportFile = base64.b64decode(payload).decode('utf-8')

reportJson = json.loads(reportFile)

# Initialized data frame to load all visuals + current title

visualsDF = pd.DataFrame({'visualID': [], 'title': []})

# Build visual + title dictionary (if exists)

for section in reportJson['sections']:

    for visual in section['visualContainers']:
        visualConfig = visual['config']
        visualConfigJson = json.loads(visualConfig)        
        visualID = visualConfigJson['name']
        try:
            title = visualConfigJson["singleVisual"]["vcObjects"]["title"][0]["properties"]["text"]["expr"]["Literal"]["Value"]
        except:
            title = ""

        if title != "":
            title = title.strip("'")
            visualsDF = pd.concat([visualsDF, pd.DataFrame({'visualID': visualID, 'title': title}, index=[0])], ignore_index=True)

visualsDF

StatementMeta(, 0b843145-b532-4aca-9aa2-62d0ffcc7add, 17, Finished, Available)

,visualID,title
0,32b08d7f32cb5f9816cc,Sales Amount by Brand and Gender
1,484fbdd73143c5bf71fa,"Sales Amount, Sales Amount (LY), Margin and Sales Amount (12M average) by Date (Year-Month)"
2,4b18758374093c2e23fb,Sales Amount vs # Customers
3,5acb1caf298449a8acb4,Sales Amount and Sales Amount (LY) by Category
4,6f9803040a3195450ea1,Value by Product
5,e50725a866c432900b35,Value by Date
6,e859ee4d7c5b10e836d6,"Value, Value (ly) and Value Daily Max by Month Start Date"
7,f0143b84500069035233,Value (ytd) by Month Start Date
8,a64038428c095bd71739,Sales Amount by Country and Gender


# Call Azure AI translator 

In [6]:
# Initialize Azure AI translator

translate = (Translate()
    .setTextCol("title")
    .setToLanguage(languages)
    .setOutputCol("translation")
    .setConcurrency(5))

# Call Azure AI translator to translate all the visual titles into multiple languages

visualsDF_spark = sqlContext.createDataFrame(visualsDF)

translatedDF = translate.transform(visualsDF_spark)\
        .withColumn("translation_result", flatten(col("translation.translations")))\
        .withColumn("translation", col("translation_result.text")[0])\
        .cache()

display(translatedDF)

translatedDF = translatedDF.toPandas()

StatementMeta(, 0b843145-b532-4aca-9aa2-62d0ffcc7add, 14, Finished, Available)

SynapseWidget(Synapse.DataFrame, 05d3e46f-1317-4bee-a906-e78573194e45)

/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:117: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Nested StructType not supported in conversion to Arrow
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.


# For each language: create a report with the translated visuals

In [10]:
for lang in languages:

    reportJsonLang = copy.deepcopy(reportJson)

    translatedReportName = f"{newReportName} - {lang}"

    print(f"Creating report '{translatedReportName}'")

    for section in reportJsonLang['sections']:

        for visual in section['visualContainers']:

            visualConfig = visual['config']
            visualConfigJson = json.loads(visualConfig)  
            visualID = visualConfigJson['name']
            
            # Find translation for the visual

            translatedTexts = translatedDF.loc[translatedDF['visualID'] == visualID, 'translation_result']
            
            if not translatedTexts.empty:

                # Get translation of the visual for the language

                translatedText = [t for t in translatedTexts.values[0] if t.to == lang][0]
                
                if translatedText is not None:
                    try:
                        
                        # Get visual title json property

                        titleProperty = visualConfigJson["singleVisual"]["vcObjects"]["title"][0]["properties"]["text"]["expr"]["Literal"]

                        translatedTextStr = translatedText.text
                    
                        titleProperty["Value"] = f"'{translatedTextStr}'"            
            
                    except KeyError:
                        pass
            
            # Replace the visual 'config' json

            visual['config'] = json.dumps(visualConfigJson)

    # Create a new report by replacing the report.json part

    # Get all parts except report.json (we will add a new one)

    jsonPartsLang = [part for part in jsonParts if part["path"] != 'report.json']

    reportPartLang = copy.deepcopy(reportPart)

    reportPartLang['payload'] = base64.b64encode(json.dumps(reportJsonLang).encode('utf-8')).decode('utf-8')

    jsonPartsLang.append(reportPartLang)

    payload = {}
    payload['displayName'] = translatedReportName
    payload['type'] = "Report"
    payload['definition'] =  {
        'parts' : jsonPartsLang
    }

    response = client.post(f"/v1/workspaces/{workspaceId}/items", json= payload)

StatementMeta(, 0b843145-b532-4aca-9aa2-62d0ffcc7add, 18, Finished, Available)

Creating report '[trans] - Sales - fr'
Creating report '[trans] - Sales - it'
Creating report '[trans] - Sales - pt'
Creating report '[trans] - Sales - ja'
